In [176]:
from functions import *

In [177]:
n = 4

# Temperature sweep

In [178]:
num_random_models = 50
num_temperatures = 20

T_range = np.geomspace(0.001, 1000, num_temperatures)

def f(T):
    '''
    Calculate spectral gaps for quantum and mismatched proposals.
    f(T) is defined to parallelize the temperature-loop.
    '''   
    delta_random_arr = np.zeros((num_random_models))
    delta_local_arr = np.zeros((num_random_models))
    delta_quantum_arr = np.zeros((num_random_models))
    delta_mismatched_quantum_arr = np.zeros((num_random_models))

    for i in range(num_random_models):
        m = RandomIsingModel(n, seed=i) # set the seed for reproducibility

        proposal_mat_random = get_proposal_mat_random(m)
        P_random = get_transition_matrix(m, T, proposal_mat_random)
        delta_random = get_delta(P_random)
        delta_random_arr[i] = delta_random

        proposal_mat_local = get_proposal_mat_local(m)
        P_local = get_transition_matrix(m, T, proposal_mat_local)
        delta_local = get_delta(P_local)
        delta_local_arr[i] = delta_local
    
        proposal_mat_quantum = get_proposal_mat_quantum_avg(m)
        P_quantum = get_transition_matrix(m, T, proposal_mat_quantum)
        delta_quantum = get_delta(P_quantum)
        delta_quantum_arr[i] = delta_quantum

        m_mismatched = RandomIsingModel(n, seed=i+num_random_models) # set the seed for reproducibility
        proposal_mat_mismatched_quantum = get_proposal_mat_quantum_layden(m_mismatched)
        P_mismatched_quantum = get_transition_matrix(m, T, proposal_mat_mismatched_quantum)
        delta_mismatched_quantum = get_delta(P_mismatched_quantum)
        delta_mismatched_quantum_arr[i] = delta_mismatched_quantum

    return delta_random_arr, delta_local_arr, delta_quantum_arr, delta_mismatched_quantum_arr

In [179]:
sweep_data = np.asarray(joblib.Parallel(n_jobs=-1)(joblib.delayed(f)(T) for T in tqdm(T_range)))
sweep_data = sweep_data.swapaxes(0,1)




100%|██████████| 20/20 [00:19<00:00,  1.00it/s]


In [180]:
delta_random_arr = sweep_data[0]
delta_local_arr = sweep_data[1]
delta_quantum_arr = sweep_data[2]
delta_mismatched_quantum_arr = sweep_data[3]

# Save the data

In [181]:
# # Save the results for long sweeps
# temperature_sweep = {'local': delta_local_arr,
#                      'uniform': delta_random_arr,
#                      'quantum': delta_quantum_arr,
#                      'mismatched': delta_mismatched_quantum_arr,
#                      'ra': delta_ra_arr}

In [182]:
# pickle.dump(temperature_sweep, open(f'data/temperature_sweep_n{n}.p','wb'))

In [183]:
# temperature_sweep = pickle.load(open(f'data/temperature_sweep_n{n}.p','rb'))
# delta_local_arr = temperature_sweep['local']
# delta_random_arr = temperature_sweep['uniform']
# delta_quantum_arr = temperature_sweep['quantum']
# delta_mismatched_quantum_arr = temperature_sweep['mismatched']

# Average and plot the results

In [184]:
# Take an average over random Ising instances
delta_random_arr_avg = np.mean(delta_random_arr, axis=1)
delta_random_arr_std = np.std(delta_random_arr, axis=1)

delta_local_arr_avg = np.mean(delta_local_arr, axis=1)
delta_local_arr_std = np.std(delta_local_arr, axis=1)

delta_quantum_arr_avg = np.mean(delta_quantum_arr, axis=1)
delta_quantum_arr_std = np.std(delta_quantum_arr, axis=1)

delta_mismatched_quantum_arr_avg = np.mean(delta_mismatched_quantum_arr, axis=1)
delta_mismatched_quantum_arr_std = np.std(delta_mismatched_quantum_arr, axis=1)

In [185]:
plt.plot(T_range, delta_random_arr_avg, '-', color='grey', label = 'Uniform')
plt.plot(T_range, delta_local_arr_avg, '-', color='blue', label = 'Local')
plt.plot(T_range, delta_quantum_arr_avg, '-', color='red', label = 'Quantum')
plt.plot(T_range, delta_mismatched_quantum_arr_avg, '-', color='orange', label = 'Mismatched quantum')

plt.fill_between(T_range, 
                 delta_random_arr_avg-delta_random_arr_std, 
                 delta_random_arr_avg+delta_random_arr_std, 
                 color='grey',
                 alpha=0.1)
plt.fill_between(T_range, 
                 delta_local_arr_avg-delta_local_arr_std, 
                 delta_local_arr_avg+delta_local_arr_std, 
                 color='blue',
                 alpha=0.1)
plt.fill_between(T_range, 
                 delta_quantum_arr_avg-delta_quantum_arr_std, 
                 delta_quantum_arr_avg+delta_quantum_arr_std, 
                 color='red',
                 alpha=0.1)
plt.fill_between(T_range, 
                 delta_mismatched_quantum_arr_avg-delta_mismatched_quantum_arr_std, 
                 delta_mismatched_quantum_arr_avg+delta_mismatched_quantum_arr_std, 
                 color='orange',
                 alpha=0.1)

plt.title(f'n = {n}')
plt.xlabel('Temperature $T$')
plt.ylabel('Absolute spectral gap $\delta$')
plt.ylim(0.01, 1.1)
plt.xlim(0.001, 1000)
plt.yscale('log')
plt.xscale('log')
plt.legend()